In [1]:
import os
import pymongo
import pandas as pd
import json
from datetime import datetime as dt
from src.features.dedup_model import load_results
from src.data.utils import load_json_as_df, create_dir
from src.data.image_utils import download_images_from_group
import src.models.nima_model as nima 



2023-01-26 13:13:58.296235: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from src.constants import (
    RAW_DIR_PATH,
    INTERIM_DIR_PATH,
    PROCESSED_DIR_PATH
)

In [3]:
DB_USERNAME = os.getenv("MONGO_USERNAME")
DB_PASSWORD = os.getenv("MONGO_PASSWORD")
DB_URL = os.getenv("MONGO_URL")
DB_NAME = os.getenv("MONGO_DB")
DB_NAME

'mlops-project'

In [4]:
client = pymongo.MongoClient(
    f"mongodb+srv://{DB_USERNAME}:{DB_PASSWORD}@{DB_URL}/?retryWrites=true&w=majority"
)
client

MongoClient(host=['ac-52gsxzj-shard-00-02.yi2z9ch.mongodb.net:27017', 'ac-52gsxzj-shard-00-01.yi2z9ch.mongodb.net:27017', 'ac-52gsxzj-shard-00-00.yi2z9ch.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-12itgl-shard-0', tls=True)

In [5]:
db = client[DB_NAME]
db.head

Collection(Database(MongoClient(host=['ac-52gsxzj-shard-00-02.yi2z9ch.mongodb.net:27017', 'ac-52gsxzj-shard-00-01.yi2z9ch.mongodb.net:27017', 'ac-52gsxzj-shard-00-00.yi2z9ch.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-12itgl-shard-0', tls=True), 'mlops-project'), 'head')

In [6]:
db.list_collection_names()


['quality_assessment_results',
 'human_results',
 'duplicated_images_model_results']

In [7]:
collectionDuplicated = db['duplicated_images_model_results']
df = pd.DataFrame(collectionDuplicated.find())

In [8]:
df.head()

,_id,date,group_id,base_image_id,duplicated_image_id,probability
0,63d267b08cf7c3ec87a15e96,2023-01-26 11:44:47.983,102292696432481,1631,1632,0.757192
1,63d267b08cf7c3ec87a15e97,2023-01-26 11:44:47.983,102292696432481,1631,1633,0.846591
2,63d267b08cf7c3ec87a15e98,2023-01-26 11:44:47.983,102292696432481,1631,1634,0.769920
3,63d267b08cf7c3ec87a15e99,2023-01-26 11:44:47.983,102292696432481,1631,1635,0.559866
4,63d267b08cf7c3ec87a15e9a,2023-01-26 11:44:47.983,102292696432481,1631,1636,0.546532


In [9]:
len(df.index)

511845

In [10]:
images_json_file_name = "images.json"
images_json_file_path = os.path.join(RAW_DIR_PATH, images_json_file_name)

In [11]:
images_df = load_json_as_df(images_json_file_path, index_col="image_id")
images_df.head()

,group_id,image_url
image_id,,
1,2224086878288,https://i.travelapi.com/hotels/57000000/569800...
2,2224086878288,https://i.travelapi.com/hotels/57000000/569800...
3,2224086878288,https://i.travelapi.com/hotels/57000000/569800...
4,2224086878288,https://i.travelapi.com/hotels/57000000/569800...
5,2224086878288,https://i.travelapi.com/hotels/57000000/569800...


In [12]:
len(images_df.index)

65200

In [13]:
images_dir_path = os.path.join(INTERIM_DIR_PATH, "images")
create_dir(images_dir_path)

In [ ]:
for group_id, df_group in df.groupby('group_id'):

    list_images = df_group['base_image_id'].to_list() +df_group['duplicated_image_id'].to_list()
    list_images = list(dict.fromkeys(list_images))

    df_images_group= pd.DataFrame (list_images, columns = ['image_id'])
    df_images_group['group_id'] = group_id

    merged_group = pd.merge(left=df_images_group, right=images_df, left_on=['group_id','image_id'], right_on=['group_id','image_id'])

    download_images_from_group(images_df, group_id, (images_dir_path ))
    



365it [00:00, 10221.27it/s]
215it [00:00, 3015.23it/s]
25it [00:00, 716.85it/s]
125it [00:00, 810.59it/s]
132it [00:00, 2119.64it/s]
40it [00:00, 1918.21it/s]
5it [00:00, 419.91it/s]
230it [00:00, 3578.47it/s]
210it [00:00, 4461.57it/s]
300it [00:00, 4985.86it/s]
235it [00:00, 3643.06it/s]
165it [00:00, 3066.57it/s]
135it [00:00, 2159.46it/s]
96it [00:00, 1872.85it/s]
145it [00:00, 1160.58it/s]
130it [00:00, 1530.08it/s]
35it [00:00, 1252.07it/s]
275it [00:00, 2807.41it/s]
115it [00:00, 2103.61it/s]
125it [00:00, 1075.45it/s]
110it [00:00, 1358.26it/s]
245it [00:00, 3567.66it/s]
805it [00:00, 1470.55it/s]
60it [00:00, 902.39it/s]
265it [00:00, 3724.19it/s]
330it [00:00, 2347.47it/s]
150it [00:00, 974.04it/s]


In [14]:
model= "MobileNet"
src = os.path.join('/app','src')
features = os.path.join(src,'features')
nima_model_folder = os.path.join(features,'nima_model')
weights = os.path.join(nima_model_folder,'weights_mobilenet_aesthetic_0.07.hdf5')



In [15]:
images_dir_path

'/app/data/interim/images'

In [16]:
list_predicted = []
for group_id, df_group in df.groupby('group_id'):
    image_folder = os.path.join(images_dir_path,group_id)
    print(group_id)
    try:
        predicted =  nima.predictImages(model,weights,image_folder)
    except:
        print(group_id+" is not defined")
    for row in predicted:
        row['group_id'] = group_id
    list_predicted = list_predicted + predicted    

102292696432481


2023-01-26 13:14:57.741005: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/app/src/models/nima_model/__init__.py:27: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  return model.predict_generator(data_generator, workers=8, use_multiprocessing=True, verbose=1)
/usr/local/lib/python3.10/site-packages/keras/engine/data_adapter.py:925: RuntimeWarning: overflow encountered in cast
  return np.array(t, dtype=backend.floatx())


6/6 [==============================] - 15s 2s/step
<class 'list'>
105233678839355
4/4 [==============================] - 9s 2s/step
<class 'list'>
109414130587157
1/1 [==============================] - 2s 2s/step
<class 'list'>
113874454336761
2/2 [==============================] - 5s 2s/step
<class 'list'>
141329095212959
3/3 [==============================] - 6s 1s/step
<class 'list'>
191394243143100
1/1 [==============================] - 3s 3s/step
<class 'list'>
194872942013291
1/1 [==============================] - 1s 1s/step
<class 'list'>
20234819560905
4/4 [==============================] - 8s 1s/step
<class 'list'>
2224086878288
4/4 [==============================] - 10s 2s/step
<class 'list'>
232727591466254
5/5 [==============================] - 14s 2s/step
<class 'list'>
238197478000118
4/4 [==============================] - 11s 2s/step
<class 'list'>
256822072976278
3/3 [==============================] - 7s 1s/step
<class 'list'>
259754146455007
3/3 [======================

/usr/local/lib/python3.10/site-packages/keras/engine/data_adapter.py:925: RuntimeWarning: invalid value encountered in cast
  return np.array(t, dtype=backend.floatx())


<class 'list'>
59718437804789
4/4 [==============================] - 9s 2s/step
<class 'list'>
74551318319906
3/3 [==============================] - 8s 2s/step
<class 'list'>
88101071551694
6/6 [==============================] - 14s 2s/step
<class 'list'>
90313767680683
2/2 [==============================] - 6s 2s/step
<class 'list'>


In [17]:
list_predicted[0]

{'image_id': '1631',
 'mean_score_prediction': 5.60638951636065,
 'group_id': '102292696432481'}

In [18]:
print(json.dumps(list_predicted, indent=2))

[
  {
    "image_id": "1631",
    "mean_score_prediction": 5.60638951636065,
    "group_id": "102292696432481"
  },
  {
    "image_id": "1632",
    "mean_score_prediction": 5.142864143301267,
    "group_id": "102292696432481"
  },
  {
    "image_id": "1633",
    "mean_score_prediction": 4.908380738153937,
    "group_id": "102292696432481"
  },
  {
    "image_id": "1634",
    "mean_score_prediction": 5.551215432868048,
    "group_id": "102292696432481"
  },
  {
    "image_id": "1635",
    "mean_score_prediction": 4.594676813456317,
    "group_id": "102292696432481"
  },
  {
    "image_id": "1636",
    "mean_score_prediction": 5.287141878007333,
    "group_id": "102292696432481"
  },
  {
    "image_id": "1637",
    "mean_score_prediction": 4.757534959644545,
    "group_id": "102292696432481"
  },
  {
    "image_id": "1638",
    "mean_score_prediction": 6.02659667759864,
    "group_id": "102292696432481"
  },
  {
    "image_id": "1639",
    "mean_score_prediction": 6.052004906422219,
    

In [19]:
results_array = [{"date": dt.utcnow(), **obj} for obj in list_predicted]
results_array

[{'date': datetime.datetime(2023, 1, 26, 13, 21, 36, 30956),
  'image_id': '1631',
  'mean_score_prediction': 5.60638951636065,
  'group_id': '102292696432481'},
 {'date': datetime.datetime(2023, 1, 26, 13, 21, 36, 30961),
  'image_id': '1632',
  'mean_score_prediction': 5.142864143301267,
  'group_id': '102292696432481'},
 {'date': datetime.datetime(2023, 1, 26, 13, 21, 36, 30962),
  'image_id': '1633',
  'mean_score_prediction': 4.908380738153937,
  'group_id': '102292696432481'},
 {'date': datetime.datetime(2023, 1, 26, 13, 21, 36, 30963),
  'image_id': '1634',
  'mean_score_prediction': 5.551215432868048,
  'group_id': '102292696432481'},
 {'date': datetime.datetime(2023, 1, 26, 13, 21, 36, 30965),
  'image_id': '1635',
  'mean_score_prediction': 4.594676813456317,
  'group_id': '102292696432481'},
 {'date': datetime.datetime(2023, 1, 26, 13, 21, 36, 30966),
  'image_id': '1636',
  'mean_score_prediction': 5.287141878007333,
  'group_id': '102292696432481'},
 {'date': datetime.date

In [20]:
len(results_array)

5568

In [21]:
db["quality_assessment_results"].insert_many(results_array)

In [22]:
db.list_collection_names()

['quality_assessment_results',
 'human_results',
 'duplicated_images_model_results']

In [23]:
quality = db['quality_assessment_results']
df_quality = pd.DataFrame(quality.find())

In [24]:
df_quality.head()

,_id,date,image_id,mean_score_prediction,group_id
0,63d27e8fb3b516a4aa731e9d,2023-01-26 13:21:36.030,1631,5.606390,102292696432481
1,63d27e8fb3b516a4aa731e9e,2023-01-26 13:21:36.030,1632,5.142864,102292696432481
2,63d27e8fb3b516a4aa731e9f,2023-01-26 13:21:36.030,1633,4.908381,102292696432481
3,63d27e8fb3b516a4aa731ea0,2023-01-26 13:21:36.030,1634,5.551215,102292696432481
4,63d27e8fb3b516a4aa731ea1,2023-01-26 13:21:36.030,1635,4.594677,102292696432481
